https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"clarencetohjw","key":"5fea27d6f06812afc47874ca12a70297"}'}

In [5]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge           COVID-19 Open Research Dataset Challenge (CORD-19)    2GB  2020-05-13 22:21:42          65474  
roche-data-science-coalition/uncover                        UNCOVER COVID-19 Challenge                          142MB  2020-04-29 22:47:23           8355  
mrmorj/political-advertisements-from-facebook               Political Advertisements from Facebook              237MB  2020-05-05 18:53:51             58  
amanarora/obesity-among-adults-by-country-19752016          Obesity among adults by country, 1975-2016          362KB  2020-05-08 07:40:50            374  
jessemostipak/animal-crossing                               Anim

In [6]:
!kaggle competitions download -c house-prices-advanced-regression-techniques
!ls

data_description.txt: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
 data_description.txt  'kaggle (2).json'   sample_data		   test.csv
'kaggle (1).json'       kaggle.json	   sample_submission.csv   train.csv


In [7]:
# import specific libraries
# load CSVs
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# This is the format in which my submission must follow
sample = pd.read_csv('sample_submission.csv')

Data Cleaning

In [9]:
# I will combine both train and test for cleaning
combined = pd.concat([train.drop(columns=['SalePrice']),test])
combined

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,None,

In [10]:
# A look at the columns with the highest percentages of null values
list_miss=[]
for col in combined.columns:
    pct_missing = np.mean(combined[col].isnull())
    list_miss.append([col, round(pct_missing*100)])
list_miss.sort(key = lambda x: -x[1])
list_miss

[['PoolQC', 100.0],
 ['MiscFeature', 96.0],
 ['Alley', 93.0],
 ['Fence', 80.0],
 ['FireplaceQu', 49.0],
 ['LotFrontage', 17.0],
 ['GarageType', 5.0],
 ['GarageYrBlt', 5.0],
 ['GarageFinish', 5.0],
 ['GarageQual', 5.0],
 ['GarageCond', 5.0],
 ['BsmtQual', 3.0],
 ['BsmtCond', 3.0],
 ['BsmtExposure', 3.0],
 ['BsmtFinType1', 3.0],
 ['BsmtFinType2', 3.0],
 ['MasVnrType', 1.0],
 ['MasVnrArea', 1.0],
 ['Id', 0.0],
 ['MSSubClass', 0.0],
 ['MSZoning', 0.0],
 ['LotArea', 0.0],
 ['Street', 0.0],
 ['LotShape', 0.0],
 ['LandContour', 0.0],
 ['Utilities', 0.0],
 ['LotConfig', 0.0],
 ['LandSlope', 0.0],
 ['Neighborhood', 0.0],
 ['Condition1', 0.0],
 ['Condition2', 0.0],
 ['BldgType', 0.0],
 ['HouseStyle', 0.0],
 ['OverallQual', 0.0],
 ['OverallCond', 0.0],
 ['YearBuilt', 0.0],
 ['YearRemodAdd', 0.0],
 ['RoofStyle', 0.0],
 ['RoofMatl', 0.0],
 ['Exterior1st', 0.0],
 ['Exterior2nd', 0.0],
 ['ExterQual', 0.0],
 ['ExterCond', 0.0],
 ['Foundation', 0.0],
 ['BsmtFinSF1', 0.0],
 ['BsmtFinSF2', 0.0],
 ['BsmtU

In [11]:
# Drop columns with a lot of missng data, for the sake of accuracy
# the threshold is 20%, if the data has more than 20% missing values, we drop it.
cols_to_drop = []
for i in range(len(list_miss)):
    if list_miss[i][1] > 20:
        cols_to_drop.append(list_miss[i][0])

df_less = combined.drop(cols_to_drop, axis=1)
df_less 

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,GasA,TA,Y,SBrkr,970,0,0,970,0.0,1.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,80,32,0,0,0,0,700,7,2006,WD,Normal


In [0]:
# for columns in the dataframe that are of dtype float64 or int64
# fill NA with the median
def fill_median(df):
    cols = df.loc[:, df.dtypes != object].columns.tolist()
    for i in cols:
        med = df[i].median()
        df[i] = df[i].fillna(med)
    return df
def fill_mean(df):
    cols = df.loc[:, df.dtypes != object].columns.tolist()
    for i in cols:
        mea = df[i].mean()
        df[i] = df[i].fillna(mea)
    return df
def fill_mode(df):
    cols = df.loc[:, df.dtypes != object].columns.tolist()
    for i in cols:
        mod = df[i].mode()
        df[i] = df[i].fillna(mod)
    return df

In [0]:
df_median = fill_median(df_less)
df_mean = fill_mean(df_less)
df_mode = fill_mode(df_less)

In [14]:
# after filling null values of non-dtype object columns with the median
# get the dtype object columns with na values
df_median.columns[df_median.isna().any()].tolist()
df_mean.columns[df_mean.isna().any()].tolist()
df_mode.columns[df_mode.isna().any()].tolist()

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'SaleType']

In [0]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def __init__(self):
        """Impute missing values.
        Columns of dtype object are imputed with the most frequent value in column.
        Columns of other types are imputed with mean of column.
        """
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
X = pd.DataFrame(df_median)
# X = pd.DataFrame(df_mean)
# X = pd.DataFrame(df_mode)
df_final = DataFrameImputer().fit_transform(X)

In [16]:
# do a check to see if there are any more columns with missing values
df_final.columns[df_final.isna().any()].tolist()

[]

In [17]:
# checking for duplicates
# bear in mind that column Id is unique, drop it before checking
df_nodups = df_final.drop('Id', axis=1).drop_duplicates()
# there were duplicate rows
print(df_final.shape)
print(df_nodups.shape)
# 2 rows are being complete duplicate observations.

(2919, 75)
(2917, 74)


In [18]:
# upon further inspection, the 2 rows are not duplicates
df_final[df_final.drop('Id', axis=1).duplicated()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1253,2714,160,FV,24.0,2544,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,Twnhs,2Story,7,5,2005,2005,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,600.0,600.0,GasA,Ex,Y,SBrkr,520,623,80,1223,0.0,0.0,2,1,2,1,Gd,4,Typ,0,Detchd,2005.0,RFn,2.0,480.0,TA,TA,Y,0,166,0,0,0,0,0,7,2006,WD,Normal
1405,2866,160,RM,24.0,2522,Pave,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,Twnhs,2Story,7,5,2004,2004,Gable,CompShg,VinylSd,VinylSd,Stone,50.0,Gd,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,970.0,970.0,GasA,Ex,Y,SBrkr,970,739,0,1709,0.0,0.0,2,0,3,1,Gd,7,Maj1,0,Detchd,2004.0,Unf,2.0,380.0,TA,TA,Y,0,40,0,0,0,0,0,5,2006,WD,Normal


In [0]:
df_numbers=df_final.loc[:, df_final.dtypes != object]
df_objects=df_final.loc[:, df_final.dtypes == object]

In [0]:
list_bins=[]
list_bin_col=[]
list_obj_col=[]
for i in df_objects.columns.tolist():
    column_values = df_objects[i].values
    unique_bins = np.unique(column_values)
    list_bins.append(np.unique(column_values))
    list_obj_col.append(i)
    for y in range(len(unique_bins)):
        list_bin_col.append(i+" "+unique_bins[y])

In [21]:
# Helper function to check for duplicates
def checkIfDuplicates_1(listOfElems):
    if len(listOfElems) == len(set(listOfElems)):
        return False
    else:
        return True
checkIfDuplicates_1(list_bin_col)
# no duplicates

False

In [22]:
# for all dtype object columns, make columns of zeros
for col in list_bin_col:
    df_final[col]=0
df_final

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,...,Functional Mod,Functional Sev,Functional Typ,GarageType 2Types,GarageType Attchd,GarageType Basment,GarageType BuiltIn,GarageType CarPort,GarageType Detchd,GarageFinish Fin,GarageFinish RFn,GarageFinish Unf,GarageQual Ex,GarageQual Fa,GarageQual Gd,GarageQual Po,GarageQual TA,GarageCond Ex,GarageCond Fa,GarageCond Gd,GarageCond Po,GarageCond TA,PavedDrive N,PavedDrive P,PavedDrive Y,SaleType COD,SaleType CWD,SaleType Con,SaleType ConLD,SaleType ConLI,SaleType ConLw,SaleType New,SaleType Oth,SaleType WD,SaleCondition Abnorml,SaleCondition AdjLand,SaleCondition Alloca,SaleCondition Family,SaleCondition Normal,SaleCondition Partial
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1457,2918,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,GasA,TA,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# re-index
df_final.index=list(range(len(df_final)))

In [0]:

def onehotenc(df,lis1):
    for y in lis1:
        col,name=y.split(" ", 1)
        for i in range(len(df)):
            if name in df[col][i]:
                df[y][i] = 1
            else:
                df[y][i] = 0
    return df

In [25]:
# One Hot Encoding
df_updated=onehotenc(df_final,list_bin_col)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [26]:
# to test one hot encoding
total_count=0
for i in range(len(df_final)):
    if 'RL' in df_final['MSZoning'][i]:
        total_count+=1
print(total_count)
df_updated['MSZoning RL'].sum()

2269


2269

In [27]:
# finally, we have done one hot encoding, but we need to remove all object dtype columns
final_df = df_updated.drop(columns=list_obj_col)
final_df.info() 
# final check that all dtypes are either float64 or int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 2918
Columns: 271 entries, Id to SaleCondition Partial
dtypes: float64(11), int64(260)
memory usage: 6.1 MB


In [28]:
# This is what the final dataframe looks like
final_df

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning C (all),MSZoning FV,MSZoning RH,...,Functional Mod,Functional Sev,Functional Typ,GarageType 2Types,GarageType Attchd,GarageType Basment,GarageType BuiltIn,GarageType CarPort,GarageType Detchd,GarageFinish Fin,GarageFinish RFn,GarageFinish Unf,GarageQual Ex,GarageQual Fa,GarageQual Gd,GarageQual Po,GarageQual TA,GarageCond Ex,GarageCond Fa,GarageCond Gd,GarageCond Po,GarageCond TA,PavedDrive N,PavedDrive P,PavedDrive Y,SaleType COD,SaleType CWD,SaleType Con,SaleType ConLD,SaleType ConLI,SaleType ConLw,SaleType New,SaleType Oth,SaleType WD,SaleCondition Abnorml,SaleCondition AdjLand,SaleCondition Alloca,SaleCondition Family,SaleCondition Normal,SaleCondition Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,1.0,0.0,2,1,3,1,8,0,2003.0,2.0,548.0,0,61,0,0,0,0,0,2,2008,0,0,0,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1976.0,2.0,460.0,298,0,0,0,0,0,0,5,2007,0,0,0,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,1.0,0.0,2,1,3,1,6,1,2001.0,2.0,608.0,0,42,0,0,0,0,0,9,2008,0,0,0,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,1.0,0.0,1,0,3,1,7,1,1998.0,3.0,642.0,0,35,272,0,0,0,0,2,2006,0,0,0,...,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,1.0,0.0,2,1,4,1,9,1,2000.0,3.0,836.0,192,84,0,0,0,0,0,12,2008,0,0,0,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,546.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,5,0,1979.0,0.0,0.0,0,0,0,0,0,0,0,6,2006,0,0,0,...,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2915,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,294.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,6,0,1970.0,1.0,286.0,0,24,0,0,0,0,0,4,2006,0,0,0,...,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
2916,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,0.0,1224.0,1224,0,0,1224,1.0,0.0,1,0,4,1,7,1,1960.0,2.0,576.0,474,0,0,0,0,0,0,9,2006,0,0,0,...,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
2917,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,575.0,912.0,970,0,0,970,0.0,1.0,1,0,3,1,6,0,1979.0,0.0,0.0,80,32,0,0,0,0,700,7,2006,0,0,0,...,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [0]:
list_final_col=final_df.columns.tolist()

In [30]:
from sklearn.model_selection import train_test_split

# Grab target as y, remove target from X
y = train.SalePrice
X = final_df[:len(train)].drop(columns=['Id'])

# Split into train, test
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.7, random_state = 17)
# Check
train_X

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning C (all),MSZoning FV,MSZoning RH,MSZoning RL,...,Functional Mod,Functional Sev,Functional Typ,GarageType 2Types,GarageType Attchd,GarageType Basment,GarageType BuiltIn,GarageType CarPort,GarageType Detchd,GarageFinish Fin,GarageFinish RFn,GarageFinish Unf,GarageQual Ex,GarageQual Fa,GarageQual Gd,GarageQual Po,GarageQual TA,GarageCond Ex,GarageCond Fa,GarageCond Gd,GarageCond Po,GarageCond TA,PavedDrive N,PavedDrive P,PavedDrive Y,SaleType COD,SaleType CWD,SaleType Con,SaleType ConLD,SaleType ConLI,SaleType ConLw,SaleType New,SaleType Oth,SaleType WD,SaleCondition Abnorml,SaleCondition AdjLand,SaleCondition Alloca,SaleCondition Family,SaleCondition Normal,SaleCondition Partial
1386,60,80.0,16692,7,5,1978,1978,184.0,790.0,469.0,133.0,1392.0,1392,1392,0,2784,1.0,0.0,3,1,5,1,12,2,1978.0,2.0,564.0,0,112,0,0,440,519,2000,7,2006,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
609,20,61.0,7943,4,5,1961,1961,192.0,903.0,0.0,126.0,1029.0,1029,0,0,1029,1.0,0.0,1,0,3,1,5,0,1961.0,1.0,261.0,64,0,39,0,0,0,0,4,2007,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
498,20,65.0,7800,5,7,1967,2004,89.0,450.0,0.0,414.0,864.0,899,0,0,899,0.0,0.0,1,0,3,1,5,0,1967.0,1.0,288.0,64,0,0,0,0,0,0,6,2009,0,0,0,1,...,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
684,60,58.0,16770,7,5,1998,1998,30.0,0.0,0.0,1195.0,1195.0,1195,644,0,1839,0.0,0.0,2,1,4,1,7,0,1998.0,2.0,486.0,0,81,0,0,0,0,0,6,2010,0,0,0,1,...,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
76,20,68.0,8475,4,7,1956,1956,0.0,228.0,0.0,724.0,952.0,952,0,0,952,0.0,0.0,1,0,2,1,4,0,1956.0,1.0,283.0,0,0,0,0,0,0,0,4,2008,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,50,51.0,10480,6,5,1936,1950,0.0,0.0,0.0,1064.0,1064.0,1166,0,473,1639,0.0,0.0,1,0,3,1,6,0,1936.0,1.0,240.0,0,0,0,0,0,0,0,3,2008,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1414,50,64.0,13053,6,7,1923,2000,0.0,0.0,0.0,833.0,833.0,1053,795,0,1848,0.0,0.0,1,1,4,1,8,1,1922.0,2.0,370.0,0,0,0,0,220,0,0,6,2008,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
143,20,78.0,10335,7,5,1999,1999,183.0,679.0,0.0,811.0,1490.0,1501,0,0,1501,1.0,0.0,2,0,3,1,6,0,1999.0,2.0,577.0,144,29,0,0,0,0,0,6,2009,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1265,160,35.0,3735,7,5,1999,1999,218.0,450.0,0.0,241.0,691.0,713,739,0,1452,1.0,0.0,2,1,3,1,6,0,1999.0,2.0,506.0,0,34,0,0,0,0,0,3,2006,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [0]:
# Check
train_y.head()

1386    250000
609     118500
498     130000
684     221000
76      135750
Name: SalePrice, dtype: int64

In [0]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_log_error
def RMSLE(y_test, predictions):
    return np.sqrt(mean_squared_log_error( y_test, predictions ))

In [47]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor(n_estimators=1000, random_state = 17)
RFR.fit(train_X,train_y)
predRF = RFR.predict(val_X)
scoreRF = RMSLE(val_y, predRF)
print('RF RMSLE:', scoreRF)

RF RMSLE: 0.1371567449477695


In [49]:
# optimal score by tuning parameters which improved test score but not the competition score.
import xgboost
LR = 0.02
cs = 0.6
md = 4
ss = 0.5
est = 1000
xgb1 = xgboost.XGBRegressor(colsample_bytree=cs,
                gamma=0,                 
                learning_rate=LR,
                max_depth=md,
                min_child_weight=1.5,
                n_estimators=est,                                                                    
                reg_alpha=0.75,
                reg_lambda=0.45,
                subsample=ss,
                seed=42)
xgb1.fit(train_X,train_y)
predXGB1 = xgb1.predict(val_X)
scoreXGB1 = RMSLE(val_y, predXGB1)
print('XGB RMSLE:',scoreXGB1)

[03:28:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB RMSLE: 0.11710685727569363


In [53]:
# this xg boost model yielded the best competition score of 0.12980
xgb = xgboost.XGBRegressor(colsample_bytree=0.4,
                gamma=0,                 
                learning_rate=0.02,
                max_depth=3,
                min_child_weight=1.5,
                n_estimators=1000,                                                                    
                reg_alpha=0.75,
                reg_lambda=0.45,
                subsample=0.6,
                seed=42)
xgb.fit(train_X,train_y)
predXGB = xgb.predict(val_X)
scoreXGB = RMSLE(val_y, predXGB)
print('XGB RMSLE:',scoreXGB)

[03:30:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB RMSLE: 0.12098046980404059


In [54]:
for_test=final_df[1460:].drop(columns=['Id'])
pred_sub = xgb.predict(for_test)
result = pd.DataFrame(pred_sub)
result.columns = ['SalePrice']
result.index=list(range(1461,len(final_df)+1))
result

,SalePrice
1461,121950.093750
1462,162529.578125
1463,185200.000000
1464,191291.937500
1465,194025.062500
...,...
2915,81312.796875
2916,79801.265625
2917,172822.390625
2918,125065.023438


In [55]:
test.index = list(range(1461,len(final_df)+1))
sub = pd.concat([test,result],axis=1)
forsub = sub[['Id','SalePrice']]
forsub.index=list(range(len(forsub)))
forsub

,Id,SalePrice
0,1461,121950.093750
1,1462,162529.578125
2,1463,185200.000000
3,1464,191291.937500
4,1465,194025.062500
...,...,...
1454,2915,81312.796875
1455,2916,79801.265625
1456,2917,172822.390625
1457,2918,125065.023438


In [0]:
forsub.to_csv('submission.csv', index=False)

In [57]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Message"

100% 21.1k/21.1k [00:01<00:00, 18.6kB/s]
Successfully submitted to House Prices: Advanced Regression Techniques